In [1]:
import sys
sys.path.append("..")

In [2]:
from keras.models import Model

Using TensorFlow backend.


In [3]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import pickle

In [4]:
from hyper_params import *
import text_encoder as te
import text_decoder as td
from data_set import *

In [5]:
import bowizer
import tfidf

In [6]:
with open('../data/norvig/big.txt') as f:
    norvig = f.read()

In [7]:
chunks = make_chunks(norvig)

In [8]:
r = np.random.RandomState(42)

In [9]:
r.shuffle(chunks)
train_text, test_text = make_train_test(chunks)

In [10]:
tm = bowizer.TokenMaker([norvig], 3000)

In [11]:
embed_h = EmbeddingHyper(tm.vocab_size + 1, 256)
conv_h = ConvHyper(256, 6, 4)
rnn_h = RnnHyper(512, is_lstm=False, is_bidirectional=True, return_sequences=False)
encoder_h = te.Hyper(embed_h, [conv_h, rnn_h])

In [12]:
dernn_h = RnnHyper(512, is_lstm=False, is_bidirectional=False, return_sequences=True, unroll=True)
dec_h = DeconvHyper(256, 6, 4)
decoder_h = td.Hyper(tm.vocab_size + 1, [dernn_h, dec_h])

In [13]:
encoder = encoder_h.make_layer()
decoder = decoder_h.make_layer()

In [14]:
def make_model(max_len):
    x = Input(shape=(max_len,), name='text_input')
    h = encoder(x)
    h = decoder(h, max_len)
    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'])

    return model

In [15]:
model64, model128, model256 = make_model(64), make_model(128), make_model(256)

In [16]:
models = {64: model64, 128: model128, 256: model256 }

In [17]:
def training_round(max_len, history=None):
    if history is None:
        initial_epoch = 0
    else:
        initial_epoch = len(history['loss'])
    train, test = bowizer.SlicedWordData.Random(train_text, test_text, 2000, max_len, 10000, r, tm)
    model = models[max_len]
    newhistory = model.fit(x=train.x, y=train.y,
                            epochs=initial_epoch+5, batch_size=16,
                            validation_data=(test.x, test.y),
                            initial_epoch=initial_epoch)
    if history is None:
        history = newhistory.history
    else:
        history = {key:history[key] + newhistory.history[key] for key in history.keys()}
    return history

In [18]:
history = None

In [19]:
for i in range(20):
    print(i)
    history = training_round(64, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 150s - loss: 5.0079 - categorical_accuracy: 0.2155 - val_loss: 4.5788 - val_categorical_accuracy: 0.2678
Epoch 2/5
10000/10000 [==============================] - 147s - loss: 4.6993 - categorical_accuracy: 0.2185 - val_loss: 4.4019 - val_categorical_accuracy: 0.2689
Epoch 3/5
10000/10000 [==============================] - 152s - loss: 4.4906 - categorical_accuracy: 0.2224 - val_loss: 4.2178 - val_categorical_accuracy: 0.2751
Epoch 4/5
10000/10000 [==============================] - 151s - loss: 4.2818 - categorical_accuracy: 0.2310 - val_loss: 4.0670 - val_categorical_accuracy: 0.2822
Epoch 5/5
10000/10000 [==============================] - 152s - loss: 4.0922 - categorical_accuracy: 0.2405 - val_loss: 3.9345 - val_categorical_accuracy: 0.2928
1
Train on 10000 samples, validate on 1000 samples
Epoch 6/10
10000/10000 [==============================] - 152s - loss: 4.0161 - categori

10000/10000 [==============================] - 153s - loss: 1.0223 - categorical_accuracy: 0.7388 - val_loss: 1.3084 - val_categorical_accuracy: 0.6956
Epoch 48/50
10000/10000 [==============================] - 145s - loss: 0.9295 - categorical_accuracy: 0.7583 - val_loss: 1.3387 - val_categorical_accuracy: 0.6921
Epoch 49/50
10000/10000 [==============================] - 145s - loss: 0.8633 - categorical_accuracy: 0.7725 - val_loss: 1.3485 - val_categorical_accuracy: 0.6933
Epoch 50/50
10000/10000 [==============================] - 146s - loss: 0.8099 - categorical_accuracy: 0.7831 - val_loss: 1.3644 - val_categorical_accuracy: 0.6930
10
Train on 10000 samples, validate on 1000 samples
Epoch 51/55
10000/10000 [==============================] - 148s - loss: 1.1121 - categorical_accuracy: 0.7230 - val_loss: 1.2014 - val_categorical_accuracy: 0.7161
Epoch 52/55
10000/10000 [==============================] - 145s - loss: 0.9119 - categorical_accuracy: 0.7654 - val_loss: 1.1871 - val_categ

10000/10000 [==============================] - 146s - loss: 0.4661 - categorical_accuracy: 0.8796 - val_loss: 0.8329 - val_categorical_accuracy: 0.8090
Epoch 95/95
10000/10000 [==============================] - 146s - loss: 0.4460 - categorical_accuracy: 0.8839 - val_loss: 0.8525 - val_categorical_accuracy: 0.8076
19
Train on 10000 samples, validate on 1000 samples
Epoch 96/100
10000/10000 [==============================] - 148s - loss: 0.6545 - categorical_accuracy: 0.8368 - val_loss: 0.7548 - val_categorical_accuracy: 0.8241
Epoch 97/100
10000/10000 [==============================] - 146s - loss: 0.5011 - categorical_accuracy: 0.8736 - val_loss: 0.7743 - val_categorical_accuracy: 0.8207
Epoch 98/100
10000/10000 [==============================] - 146s - loss: 0.4672 - categorical_accuracy: 0.8812 - val_loss: 0.7929 - val_categorical_accuracy: 0.8192
Epoch 99/100
10000/10000 [==============================] - 146s - loss: 0.4425 - categorical_accuracy: 0.8864 - val_loss: 0.8270 - val_c

In [20]:
history = None
for i in range(20):
    print(i)
    history = training_round(128, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 251s - loss: 4.1232 - categorical_accuracy: 0.2987 - val_loss: 3.6595 - val_categorical_accuracy: 0.3513
Epoch 2/5
10000/10000 [==============================] - 245s - loss: 3.4997 - categorical_accuracy: 0.3338 - val_loss: 3.3463 - val_categorical_accuracy: 0.3711
Epoch 3/5
10000/10000 [==============================] - 245s - loss: 3.2139 - categorical_accuracy: 0.3580 - val_loss: 3.1617 - val_categorical_accuracy: 0.3876
Epoch 4/5
10000/10000 [==============================] - 246s - loss: 3.0069 - categorical_accuracy: 0.3799 - val_loss: 3.0079 - val_categorical_accuracy: 0.4076
Epoch 5/5
10000/10000 [==============================] - 246s - loss: 2.8185 - categorical_accuracy: 0.4057 - val_loss: 2.8622 - val_categorical_accuracy: 0.4294
1
Train on 10000 samples, validate on 1000 samples
Epoch 6/10
10000/10000 [==============================] - 250s - loss: 2.7870 - categori

10000/10000 [==============================] - 247s - loss: 1.0248 - categorical_accuracy: 0.7377 - val_loss: 1.2608 - val_categorical_accuracy: 0.7066
Epoch 48/50
10000/10000 [==============================] - 247s - loss: 0.9643 - categorical_accuracy: 0.7499 - val_loss: 1.2589 - val_categorical_accuracy: 0.7075
Epoch 49/50
10000/10000 [==============================] - 247s - loss: 0.9192 - categorical_accuracy: 0.7595 - val_loss: 1.2599 - val_categorical_accuracy: 0.7093
Epoch 50/50
10000/10000 [==============================] - 247s - loss: 0.8827 - categorical_accuracy: 0.7666 - val_loss: 1.2656 - val_categorical_accuracy: 0.7073
10
Train on 10000 samples, validate on 1000 samples
Epoch 51/55
10000/10000 [==============================] - 251s - loss: 1.0649 - categorical_accuracy: 0.7302 - val_loss: 1.1978 - val_categorical_accuracy: 0.7199
Epoch 52/55
10000/10000 [==============================] - 247s - loss: 0.9364 - categorical_accuracy: 0.7576 - val_loss: 1.2340 - val_categ

10000/10000 [==============================] - 248s - loss: 0.5995 - categorical_accuracy: 0.8354 - val_loss: 0.9804 - val_categorical_accuracy: 0.7705
Epoch 95/95
10000/10000 [==============================] - 248s - loss: 0.5776 - categorical_accuracy: 0.8402 - val_loss: 1.0209 - val_categorical_accuracy: 0.7624
19
Train on 10000 samples, validate on 1000 samples
Epoch 96/100
10000/10000 [==============================] - 252s - loss: 0.7574 - categorical_accuracy: 0.8007 - val_loss: 0.9180 - val_categorical_accuracy: 0.7812
Epoch 97/100
10000/10000 [==============================] - 247s - loss: 0.6395 - categorical_accuracy: 0.8284 - val_loss: 0.9461 - val_categorical_accuracy: 0.7762
Epoch 98/100
10000/10000 [==============================] - 247s - loss: 0.6078 - categorical_accuracy: 0.8349 - val_loss: 0.9660 - val_categorical_accuracy: 0.7751
Epoch 99/100
10000/10000 [==============================] - 248s - loss: 0.5856 - categorical_accuracy: 0.8391 - val_loss: 0.9668 - val_c

In [21]:
history = None
for i in range(20):
    print(i)
    history = training_round(64, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 148s - loss: 1.2960 - categorical_accuracy: 0.7294 - val_loss: 0.8698 - val_categorical_accuracy: 0.7990
Epoch 2/5
10000/10000 [==============================] - 146s - loss: 0.5858 - categorical_accuracy: 0.8522 - val_loss: 0.7970 - val_categorical_accuracy: 0.8182
Epoch 3/5
10000/10000 [==============================] - 146s - loss: 0.5095 - categorical_accuracy: 0.8702 - val_loss: 0.7729 - val_categorical_accuracy: 0.8226
Epoch 4/5
10000/10000 [==============================] - 146s - loss: 0.4750 - categorical_accuracy: 0.8780 - val_loss: 0.8102 - val_categorical_accuracy: 0.8190
Epoch 5/5
10000/10000 [==============================] - 146s - loss: 0.4551 - categorical_accuracy: 0.8825 - val_loss: 0.8225 - val_categorical_accuracy: 0.8175
1
Train on 10000 samples, validate on 1000 samples
Epoch 6/10
10000/10000 [==============================] - 148s - loss: 0.6151 - categori

10000/10000 [==============================] - 146s - loss: 0.3471 - categorical_accuracy: 0.9183 - val_loss: 0.5293 - val_categorical_accuracy: 0.8802
Epoch 48/50
10000/10000 [==============================] - 146s - loss: 0.3343 - categorical_accuracy: 0.9199 - val_loss: 0.5590 - val_categorical_accuracy: 0.8741
Epoch 49/50
10000/10000 [==============================] - 146s - loss: 0.3253 - categorical_accuracy: 0.9211 - val_loss: 0.6045 - val_categorical_accuracy: 0.8663
Epoch 50/50
10000/10000 [==============================] - 146s - loss: 0.3190 - categorical_accuracy: 0.9225 - val_loss: 0.5861 - val_categorical_accuracy: 0.8715
10
Train on 10000 samples, validate on 1000 samples
Epoch 51/55
10000/10000 [==============================] - 148s - loss: 0.4444 - categorical_accuracy: 0.8925 - val_loss: 0.5325 - val_categorical_accuracy: 0.8793
Epoch 52/55
10000/10000 [==============================] - 146s - loss: 0.3381 - categorical_accuracy: 0.9205 - val_loss: 0.5350 - val_categ

10000/10000 [==============================] - 146s - loss: 0.2914 - categorical_accuracy: 0.9321 - val_loss: 0.5413 - val_categorical_accuracy: 0.8845
Epoch 95/95
10000/10000 [==============================] - 146s - loss: 0.2898 - categorical_accuracy: 0.9317 - val_loss: 0.5495 - val_categorical_accuracy: 0.8824
19
Train on 10000 samples, validate on 1000 samples
Epoch 96/100
10000/10000 [==============================] - 148s - loss: 0.4038 - categorical_accuracy: 0.9045 - val_loss: 0.4762 - val_categorical_accuracy: 0.8954
Epoch 97/100
10000/10000 [==============================] - 146s - loss: 0.3036 - categorical_accuracy: 0.9303 - val_loss: 0.4911 - val_categorical_accuracy: 0.8931
Epoch 98/100
10000/10000 [==============================] - 146s - loss: 0.2943 - categorical_accuracy: 0.9316 - val_loss: 0.5272 - val_categorical_accuracy: 0.8853
Epoch 99/100
10000/10000 [==============================] - 146s - loss: 0.2908 - categorical_accuracy: 0.9314 - val_loss: 0.5411 - val_c

In [22]:
model64.save('../models/wl_model64')